In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score
from tqdm import tqdm

In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [3]:
# 1. Load and Preprocess Data
def load_and_preprocess_data(file_path):
    # Load data
    df = pd.read_csv(file_path)
    
    # Convert Timestamp to datetime and set as index
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)


    df.drop(columns=['Knock', 'IgnitionTiming'], inplace=True)
    
    # Normalize data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)
    
    return df, scaled_data, scaler

In [4]:
# Step 1: Load and preprocess data
file_path = '../data/engine_knock_data_hourly.csv'  
df_resampled, scaled_data, scaler = load_and_preprocess_data(file_path)

In [5]:
df_resampled

,RPM,CylinderPressure,BurnRate,Vibration,EGOVoltage,TempSensor
Timestamp,,,,,,
2024-01-01 00:00:00,3049.671415,30.226362,8.532683,0.493748,0.30,106.963896
2024-01-01 01:00:00,3115.583092,20.206043,7.773047,0.005990,0.45,100.848934
2024-01-01 02:00:00,3314.768854,22.896396,9.217628,0.116642,0.45,110.003050
2024-01-01 03:00:00,3505.856376,12.276180,0.814754,0.077569,0.45,92.596056
2024-01-01 04:00:00,3409.597364,16.095535,6.928412,0.069932,0.45,91.748085
...,...,...,...,...,...,...
2025-03-31 19:00:00,2583.022048,18.871620,9.923854,-0.014610,0.45,95.126803
2025-03-31 20:00:00,2455.293210,21.390331,7.016738,-0.099423,0.45,103.119660
2025-03-31 21:00:00,2541.669115,22.035972,9.061003,0.077802,0.45,109.012643


In [6]:
scaled_data

array([[5.35129955e-01, 8.89685456e-01, 8.48549983e-01, 8.42716887e-01,
        8.71714478e-10, 7.51700950e-01],
       [5.75842515e-01, 5.75900390e-01, 7.73006403e-01, 2.33264474e-01,
        1.00000000e+00, 5.98006873e-01],
       [6.98876287e-01, 6.60148466e-01, 9.16665773e-01, 3.71523920e-01,
        1.00000000e+00, 8.28087321e-01],
       ...,
       [2.21345282e-01, 6.33204399e-01, 9.01089837e-01, 3.22994100e-01,
        9.99999999e-01, 8.03194358e-01],
       [3.92518428e-01, 5.83313640e-01, 5.47595603e-01, 2.36903540e-01,
        1.00000000e+00, 6.72546491e-01],
       [4.03071009e-01, 3.79071491e-01, 8.28537686e-02, 3.34769279e-01,
        9.99999999e-01, 4.35665354e-01]])

In [7]:
scaled_data.shape

(10944, 6)

In [8]:
# Convert to tensor
data_tensor = torch.tensor(scaled_data, dtype=torch.float32)
data_tensor

tensor([[5.3513e-01, 8.8969e-01, 8.4855e-01, 8.4272e-01, 8.7171e-10, 7.5170e-01],
        [5.7584e-01, 5.7590e-01, 7.7301e-01, 2.3326e-01, 1.0000e+00, 5.9801e-01],
        [6.9888e-01, 6.6015e-01, 9.1667e-01, 3.7152e-01, 1.0000e+00, 8.2809e-01],
        ...,
        [2.2135e-01, 6.3320e-01, 9.0109e-01, 3.2299e-01, 1.0000e+00, 8.0319e-01],
        [3.9252e-01, 5.8331e-01, 5.4760e-01, 2.3690e-01, 1.0000e+00, 6.7255e-01],
        [4.0307e-01, 3.7907e-01, 8.2854e-02, 3.3477e-01, 1.0000e+00, 4.3567e-01]])

In [9]:
# 2. Create Sequences for Training
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length
    
    def __len__(self):
        return len(self.data) - self.seq_length
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_length]
        y = self.data[idx + self.seq_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [10]:
# Step 2: Create dataset and dataloader
seq_length = 72  # Number of timesteps to look back (3 days look back)
dataset = TimeSeriesDataset(data_tensor, seq_length)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

### LSTM

In [11]:
# 3. Build LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [12]:
# Step 3: Define model, loss, and optimizer
input_size = df_resampled.shape[1]  # Number of features
hidden_size = 128
num_layers = 1
output_size = df_resampled.shape[1]

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
output_size

6

In [14]:
model

LSTMModel(
  (lstm): LSTM(6, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

### LSTM + Attention

In [15]:
# LSTM + Attention
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x: [batch_size, seq_len, hidden_size]
        attn_weights = torch.softmax(self.attn(x), dim=1)  # [batch_size, seq_len, 1]
        context = torch.sum(attn_weights * x, dim=1)  # [batch_size, hidden_size]
        return context

class LSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMWithAttention, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = Attention(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.attention(out)
        out = self.fc(out)
        return out
    

## Params
hidden_size = 64
num_layers = 3

lstm_attn_model = LSTMWithAttention(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
lstm_attn_optimizer = torch.optim.Adam(lstm_attn_model.parameters(), lr=0.001)

In [ ]:
lstm_attn_model

### Transformer

In [ ]:
# Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_encoder_layers, output_size):
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Linear(input_size, d_model)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead),
            num_layers=num_encoder_layers
        )
        self.fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        x = self.embedding(x)  # [batch_size, seq_len, d_model]
        x = self.transformer(x)  # [batch_size, seq_len, d_model]
        x = self.fc(x[:, -1, :])  # Take last token's output
        return x
    

# Define model
input_size = df_resampled.shape[1]  # Number of features
d_model = 64
nhead = 8
num_encoder_layers = 3
output_size = df_resampled.shape[1]

trans_model = TransformerModel(input_size, d_model, nhead, num_encoder_layers, output_size)
criterion = nn.MSELoss()
trans_optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
trans_model

### Model Training

In [22]:
# 4. Train the Model
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        all_y_true = []
        all_y_pred = []
        
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for X_batch, y_batch in pbar:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            # Collect true and predicted values
            y_true = y_batch.cpu().numpy()
            y_pred = outputs.detach().cpu().numpy()
            
            all_y_true.extend(y_true)
            all_y_pred.extend(y_pred)
        
        # After processing all batches in the epoch
        avg_loss = total_loss / len(dataloader)
        r2 = r2_score(all_y_true, all_y_pred, multioutput='variance_weighted')
        
        print(f"\nEpoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}, R²: {r2:.4f}")

In [23]:
# 5. Forecast Future Values
def forecast(model, last_sequence, n_steps, scaler):
    model.eval()
    predictions = []
    with torch.no_grad():
        for _ in range(n_steps):
            pred = model(last_sequence)
            predictions.append(pred.numpy()[0])
            # Update the sequence by appending the prediction and removing the first element
            last_sequence = torch.cat([last_sequence[:, 1:], pred.unsqueeze(0)], dim=1)
    
    # Inverse transform the predictions
    forecasted_values = scaler.inverse_transform(np.array(predictions))
    return forecasted_values

In [24]:
def evaluate_model(model, dataloader, scaler):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            
            # Append actual and predicted values
            actuals.append(y_batch.cpu().numpy())
            predictions.append(outputs.cpu().numpy())
    
    # Concatenate all predictions and actuals
    actuals = np.concatenate(actuals, axis=0)
    predictions = np.concatenate(predictions, axis=0)
    
    # Inverse transform the predictions and actuals
    actuals_inv = scaler.inverse_transform(actuals)
    predictions_inv = scaler.inverse_transform(predictions)
    
    # Calculate R² Score
    r2 = r2_score(actuals_inv, predictions_inv, multioutput='variance_weighted')
    print(f"R² Score: {r2:.4f}")

In [ ]:
# Step 4: Train the model
num_epochs = 25
train_model(model, dataloader, criterion, optimizer, num_epochs)

In [ ]:
evaluate_model(model, dataloader, scaler)

In [ ]:
# Step 5: Forecast future values
last_seq = data_tensor[-seq_length:]
last_seq = last_seq.unsqueeze(0)  # Add batch dimension
n_steps = 720  # Forecast next 720 steps (30 dayss)

forecasted_values = forecast(model, last_seq, n_steps, scaler)

# Create a DataFrame for forecasted values
forecast_df = pd.DataFrame(
    forecasted_values,
    columns=df_resampled.columns,
    index=pd.date_range(
        start=df_resampled.index[-1] + pd.Timedelta(hours=1),
        periods=n_steps,
        freq='H'
    )
)

In [ ]:
forecast_df

In [30]:
import matplotlib.pyplot as plt

In [ ]:
# Create subplots for each feature
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('Forecasted Values for Next 30 Days')

# Flatten axes for easier iteration
axes = axes.flatten()

# Plot each feature
for idx, column in enumerate(forecast_df.columns):
    axes[idx].plot(forecast_df.index, forecast_df[column])
    axes[idx].set_title(column)
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True)
    # Rotate x-axis labels for better readability
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Create subplots for each feature
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('Historical vs Forecasted Values')

# Flatten axes for easier iteration
axes = axes.flatten()

# Get last 30 days of historical data
last_30_days = df_resampled.last('30D')

# Plot each feature
for idx, column in enumerate(forecast_df.columns):
    # Plot historical data
    axes[idx].plot(last_30_days.index, last_30_days[column], label='Historical', color='blue')
    # Plot forecasted data
    axes[idx].plot(forecast_df.index, forecast_df[column], label='Forecast', color='red')
    axes[idx].set_title(column)
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True)
    axes[idx].legend()
    # Rotate x-axis labels for better readability
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [33]:
# # Print or save the forecasted DataFrame
forecast_df.to_csv('forecasted_data_1.csv', index_label='Timestamp')

In [ ]:
train_model(lstm_attn_model, dataloader, criterion, lstm_attn_optimizer, num_epochs)

In [ ]:
evaluate_model(lstm_attn_model, dataloader, scaler)

In [ ]:
# Step 5: Forecast future values
last_seq = data_tensor[-seq_length:]
last_seq = last_seq.unsqueeze(0)  # Add batch dimension
n_steps = 720  # Forecast next 720 steps (30 dayss)

forecasted_values = forecast(lstm_attn_model, last_seq, n_steps, scaler)

# Create a DataFrame for forecasted values
forecast_df = pd.DataFrame(
    forecasted_values,
    columns=df_resampled.columns,
    index=pd.date_range(
        start=df_resampled.index[-1] + pd.Timedelta(hours=1),
        periods=n_steps,
        freq='H'
    )
)

In [ ]:
# Create subplots for each feature
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('Forecasted Values for Next 30 Days')

# Flatten axes for easier iteration
axes = axes.flatten()

# Plot each feature
for idx, column in enumerate(forecast_df.columns):
    axes[idx].plot(forecast_df.index, forecast_df[column])
    axes[idx].set_title(column)
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True)
    # Rotate x-axis labels for better readability
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Create subplots for each feature
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('Historical vs Forecasted Values')

# Flatten axes for easier iteration
axes = axes.flatten()

# Get last 30 days of historical data
last_30_days = df_resampled.last('30D')

# Plot each feature
for idx, column in enumerate(forecast_df.columns):
    # Plot historical data
    axes[idx].plot(last_30_days.index, last_30_days[column], label='Historical', color='blue')
    # Plot forecasted data
    axes[idx].plot(forecast_df.index, forecast_df[column], label='Forecast', color='red')
    axes[idx].set_title(column)
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True)
    axes[idx].legend()
    # Rotate x-axis labels for better readability
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# # Print or save the forecasted DataFrame
# print(forecast_df.head())
# forecast_df.to_csv('forecasted_data.csv', index_label='Timestamp')

In [ ]:
train_model(trans_model, dataloader, criterion, trans_optimizer, num_epochs)

In [ ]:
evaluate_model(trans_model, dataloader, scaler)

In [18]:
# # Step 5: Forecast future values
# last_seq = data_tensor[-seq_length:]
# last_seq = last_seq.unsqueeze(0)  # Add batch dimension
# n_steps = 720  # Forecast next 720 steps (1 hour at 1-minute intervals)

# forecasted_values = forecast(model, last_seq, n_steps, scaler)

# # Create a DataFrame for forecasted values
# forecast_df = pd.DataFrame(
#     forecasted_values,
#     columns=df_resampled.columns,
#     index=pd.date_range(
#         start=df_resampled.index[-1] + pd.Timedelta(hours=1),
#         periods=n_steps,
#         freq='T'
#     )
# )



In [19]:
# # Print or save the forecasted DataFrame
# print(forecast_df.head())
# forecast_df.to_csv('forecasted_data.csv', index_label='Timestamp')